In [1]:
#Imports---------------------------------------------------------------
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

import glob
import numpy as np
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
#Main pipeline for multiclass classification
#import libraries
import numpy as np
from time import time
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm
from sklearn.model_selection import KFold
from torchvision import models, transforms
import pickle
import os
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
import torch.nn.functional as F

K=10
arc='wideresnet'
gamma=1 #Loss Parameters
thresh=0.95 #Threshold
mu=7 #Parameter of unlabeled vs labeled --batch size
lr=0.03
b_s=32
ep=50
init=50 #Number of epochs that start training the model

PATH='/content/drive/My Drive/ML/model.pth'
PATH2='/content/drive/My Drive/ML/files/unl.pth'


In [4]:
##Dataset CIFAR 10 Feito por mim-------------------------------------------------------------------------------
##----------------------------------------------------------------------------------------------------------------------------

transform = transforms.Compose([ 
 transforms.RandomHorizontalFlip(),
 transforms.RandomVerticalFlip()])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)


counter=[0]*10
labset=[]
labset_l=[]
unlabset=[]
for i in range (0,50000):
  for z in range(0,10):
    if(trainset[i][1]==z):
      if(counter[z]<25):
        counter[z]=counter[z]+1
        labset.append(trainset[i])

      else:
        unlabset.append(trainset[i])  
print(counter)
print(len(labset))
print(len(unlabset)) 

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
[25, 25, 25, 25, 25, 25, 25, 25, 25, 25]
250
49750


In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_strong = transforms.Compose([
 #transforms.Resize(224),                    #[2]
 #transforms.CenterCrop(224),                #[3]
 torchvision.transforms.RandAugment(2,6),
 transforms.ToTensor(),                     #[4]
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

b_s=32
unlabset
train_lab=[]
train_w=[]
train_s=[]
for i in range(0,250):
  train_lab.append([transform(labset[i][0]),labset[i][1]])
for i in range(0,49750):  
  train_w.append([transform(unlabset[i][0]),unlabset[i][1]])  
  train_s.append([transform_strong(unlabset[i][0]),unlabset[i][1]])
labloader = torch.utils.data.DataLoader(train_lab, batch_size=b_s,shuffle=True, num_workers=2)
unlabloader_w = torch.utils.data.DataLoader(train_w, batch_size=b_s*mu,shuffle=False, num_workers=2)
unlabloader_s = torch.utils.data.DataLoader(train_s, batch_size=b_s*mu,shuffle=False, num_workers=2)


In [6]:
##Wideresnet usada por eles-------------------------------------------------------------------------------
##----------------------------------------------------------------------------------------------------------------------------
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F

logger = logging.getLogger(__name__)


def mish(x):
    """Mish: A Self Regularized Non-Monotonic Neural Activation Function (https://arxiv.org/abs/1908.08681)"""
    return x * torch.tanh(F.softplus(x))


class PSBatchNorm2d(nn.BatchNorm2d):
    """How Does BN Increase Collapsed Neural Network Filters? (https://arxiv.org/abs/2001.11216)"""

    def __init__(self, num_features, alpha=0.1, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True):
        super().__init__(num_features, eps, momentum, affine, track_running_stats)
        self.alpha = alpha

    def forward(self, x):
        return super().forward(x) + self.alpha


class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, drop_rate=0.0, activate_before_residual=False):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes, momentum=0.001)
        self.relu1 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes, momentum=0.001)
        self.relu2 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.drop_rate = drop_rate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                                                                padding=0, bias=False) or None
        self.activate_before_residual = activate_before_residual

    def forward(self, x):
        if not self.equalInOut and self.activate_before_residual == True:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.drop_rate > 0:
            out = F.dropout(out, p=self.drop_rate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)


class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, drop_rate=0.0, activate_before_residual=False):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(
            block, in_planes, out_planes, nb_layers, stride, drop_rate, activate_before_residual)

    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, drop_rate, activate_before_residual):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes,
                                i == 0 and stride or 1, drop_rate, activate_before_residual))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)


class WideResNet(nn.Module):
    def __init__(self, num_classes, depth=28, widen_factor=2, drop_rate=0.0):
        super(WideResNet, self).__init__()
        channels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) / 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, channels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(
            n, channels[0], channels[1], block, 1, drop_rate, activate_before_residual=True)
        # 2nd block
        self.block2 = NetworkBlock(
            n, channels[1], channels[2], block, 2, drop_rate)
        # 3rd block
        self.block3 = NetworkBlock(
            n, channels[2], channels[3], block, 2, drop_rate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(channels[3], momentum=0.001)
        self.relu = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.fc = nn.Linear(channels[3], num_classes)
        self.channels = channels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='leaky_relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1.0)
                nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0.0)

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(-1, self.channels)
        return self.fc(out)


In [9]:
#Give to the model the architecture and number of classes (K)
model = WideResNet(K)
#put the model on 'GPU' or 'CPU'
model = model.to(device)
#Define optimizer and scheduler 
optimizer = optim.Adam(model.parameters(),lr )

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)

In [ ]:
running_loss=1010.0

for epoch in range(100):
    print('\n')
    print('Iteração:',epoch)
    print('loss:',running_loss)
    #print('acc:',avg_acc)
    running_loss=0
    end=0
    count=1
    unlabeled_iter_w = iter(unlabloader_w)
    unlabeled_iter_s = iter(unlabloader_s)
    labeled_iter = iter(labloader)

    if(epoch<=init):     
     labeled_iter = iter(labloader)
    else:
      scheduler.step(running_loss)             
       
    while (end==0):
     if(epoch>init): 
        #Load labeled Images 
       try:
         inputs, labels = labeled_iter.next()
       except StopIteration:
        labeled_iter = iter(labloader)
        inputs, labels = labeled_iter.next()

       #Load unlabeled Images    
       try:  
         weak,_ = unlabeled_iter_w.next()
         strong,_ = unlabeled_iter_s.next()
       except StopIteration:
         end=1  
      
       if(end==0):  
         inputs,labels = inputs.to(device),labels.to(device)
         weak,strong = weak.to(device),strong.to(device)
         optimizer.zero_grad()
         logits_x = model(inputs)
         logits_u_w, logits_u_s = model(weak),model(strong)

          #Calculate Loss for the labeled
         Lx = F.cross_entropy(logits_x, labels, reduction='mean')    

        #Unlabeled data----------------------------------------------------
         pseudo_label =  torch.softmax(logits_u_s,dim=-1)
         pseudo_label,index = torch.max(pseudo_label,dim=-1)
         mask = pseudo_label.ge(thresh).float()
         Lu = (F.cross_entropy(logits_u_s, index,reduction='none') * mask).mean()
     
        #Calculate both losses------------------------------------------------
         loss = Lx + gamma*Lu
         loss.backward()
         optimizer.step()     
         running_loss += loss.item()
         print('.',end='')

     else:
        count=count+1
        if(count==112):
          end=1
     
        try:
          inputs, labels = labeled_iter.next()
          optimizer.zero_grad()

        except StopIteration:
          end=1
        if(end==0):  
          inputs,labels = inputs.to(device),labels.to(device)
          
          #labeled data---------------------------------------------
          outputs = model(inputs)  
          Lx = F.cross_entropy(outputs, labels, reduction='mean')    
          loss = Lx #+ gama*Lu
          loss.backward()
          optimizer.step()     
          
          running_loss += loss.item()
          print('.',end='')

              



Iteração: 0
loss: 1010.0
........

Iteração: 1
loss: 21.51690101623535
........

Iteração: 2
loss: 19.078775644302368
........

Iteração: 3
loss: 17.80989146232605
........

Iteração: 4
loss: 16.98788285255432
........

Iteração: 5
loss: 16.36384117603302
........

Iteração: 6
loss: 16.237685680389404
........

Iteração: 7
loss: 16.150359272956848
........

Iteração: 8
loss: 16.215304255485535
........

Iteração: 9
loss: 16.097626566886902
........

Iteração: 10
loss: 15.848164558410645
........

Iteração: 11
loss: 15.493884563446045
........

Iteração: 12
loss: 15.482427477836609
........

Iteração: 13
loss: 15.467018365859985
........

Iteração: 14
loss: 15.201375246047974
........

Iteração: 15
loss: 15.366220831871033
........

Iteração: 16
loss: 15.541554927825928
........

Iteração: 17
loss: 15.67035722732544
........

Iteração: 18
loss: 15.16171407699585
........

Iteração: 19
loss: 15.055948138237
........

Iteração: 20
loss: 14.71842622756958
........

Iteração: 21
loss: 14.

In [2]:
running_loss=0.0
train_acc=0.0
final_t_a=0.0
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
l= len(testloader)*32
avg_acc=0.0


##Test the dataset
model.eval()
for inputs,labels in testloader:
          inputs,labels = inputs.to(device),labels.to(device)
          #weak,strong = weak.to(device),strong.to(device)

          #labeled data---------------------------------------------
          outputs = model(inputs)  
          Lx = F.cross_entropy(outputs, labels, reduction='mean')    
          #loss_l = loss_func(outputs, labels)
          _, predicted = torch.max(outputs, 1)
          real=labels
          correct = (predicted == real).float().sum()
          avg_acc +=correct*100/l
          print(len(corrected))
          loss = Lx #+ gama*Lu
          #loss = loss_l + gama*loss_u
          #loss.backward()
          #optimizer.step()     
          running_loss += loss.item() 
          print('.',end='')

print('\n')
print('loss:',running_loss)
print('acc:',avg_acc)
print('Finished Validation')

NameError: ignored